In [52]:
from logging import exception
from conct_db import enginex
import pandas as pd
import numpy as np
import datetime
import urllib
from datetime import date



In [53]:
# from sqlalchemy.types import Integer
from sqlalchemy.types import DATE
# from sqlalchemy.types import VARCHAR 
# from sqlalchemy.types import INTEGER
# from sqlalchemy.types import Text
# from sqlalchemy.types import DATETIME

In [55]:
# parameters = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
#                                     "SERVER=WIN-SERVIDOR-BD\DWGRUPOTOTAL;"
#                                     "DATABASE=DW_Central;"
#                                     "UID=sa;"
#                                     "PWD=DWpln21")
periodo = datetime.datetime.now()

**UTILIZAR EL METODO CONNECT PARA EJECUTAR COSNULTAS**


Utilizo el text= para construir el query y ejecutarlo

In [56]:
# text=("""SELECT *
#         FROM BEES_Algoritmo
#         WHERE Periodo = '2022-07-01'""")
# with enginex.connect() as conn:
#     # conn.execute(text)
#     conn.exec_driver_sql(text)



**PARA BORRAR LOS DATOS DEL MES EN CURSO**

In [57]:
# text=("""DELETE BEES_Algoritmo
#     WHERE Periodo = '2022-07-01'""")
# enginex.execute(text)

**PARA LEER DESDE LA BASE DE DATOS**

In [58]:
# text=("""SELECT *
#         FROM BEES_Algoritmo
#         WHERE Periodo = '2022-07-01'""")
# 
# dataframendo= pd.DataFrame(enginex.execute(text))
# dataframendo.head(3)

In [59]:
df_original = pd.read_excel("y:/19-POWER BI/01-Total Bebidas/TACO GQ/Algoritmo_Agosto.xlsx", sheet_name='Export', skipfooter=3)
df = pd.DataFrame(df_original)
df['POC_ID'] = df['POC_ID'].astype('string')
# df.dtypes
# df.sample(10)

In [60]:
# if df.POC_ID.str.extract('(\d{6,6}(?:\.\d+)?)') == '704250':
#     df['COD_DISTRI'] = '70425'
# else:
#     df['COD_DISTRI'] = df.POC_ID.str.extract('(\d{6,6}(?:\.\d+)?)')

In [61]:
df['COD_CLIENTE'] = df.POC_ID.str.extract('(0{2,}\d+)')
df['COD_DISTRI'] = df.POC_ID.str.extract('(\d{6,6}(?:\.\d+)?)')
df = df[df.COD_DISTRI.isin(['704250','328660','369117','369846','445625'])]
df['COD_DISTRI'] = df['COD_DISTRI'].astype('Int64')
df['COD_DISTRI'] = np.where(df['COD_DISTRI']==704250, 70425, df['COD_DISTRI'])
df['COD_CLIENTE'] = df['COD_CLIENTE'].astype('Int64')
df['Periodo'] = pd.to_datetime(date(periodo.year, periodo.month, 1))

# df.nunique()
# df.sample(5)

In [64]:
# df['Periodo'] = df[['Periodo']].astype(Date)
df.dtypes

POC_ID                           string
Ordenes Influenciadas             int64
Ord inf. Oportunidad              int64
Ord inf.Pedido facil              int64
Ordenes totales                   int64
COD_CLIENTE                       Int64
COD_DISTRI                       object
Periodo                  datetime64[ns]
dtype: object

In [43]:
# df['Periodo'] = pd.to_datetime('1/6/2022')
# df.convert_dtypes()
# df['POC_ID'].astype('Int64')
# df.dtypes

In [44]:
# conditionlist = [
#     (df['POC_ID']>=7042500000000) & (df['POC_ID']<7042600000000),
#     (df['POC_ID']>=32866000000000) & (df['POC_ID']<32866100000000),
#     (df['POC_ID']>=36911700000000) & (df['POC_ID']<36911800000000),
#     (df['POC_ID']>=36984600000000) & (df['POC_ID']<36984700000000),
#     (df['POC_ID']>=44562500000000) & (df['POC_ID']<44562600000000)]
# choicelist = ['70425', '328660', '369117', '369846', '445625']
# df['DISTRI'] = np.select(conditionlist, choicelist, default = 'OTRO DISTRI')
# df.sample

In [65]:
df.rename({'Ord inf.Pedido facil':'Ordenes inf.Quick Order', 'Ord inf. Oportunidad':'Ordenes inf. Upsell'}, axis=1, inplace=True)

**BORRAMOS LAS COLUMNAS INECESARIAS Y REORDENAMOS EL DATAFRAME**

In [66]:
try: 
    df.drop(columns=['POC_ID'], inplace=True)
    # df = df[['Periodo', 'COD_DISTRI', 'COD_CLIENTE', 'Ordenes inf.Quick Order', 'Ordenes inf. Upsell', 'Ordenes Influenciadas', 'Ordenes totales',]]
except Exception as ex: 
    print("Oh rallos y centellas, a ocurrido un error catastrofico! ", ex)
try:
    df = df[['Periodo', 'COD_DISTRI', 'COD_CLIENTE', 'Ordenes inf.Quick Order', 'Ordenes inf. Upsell', 'Ordenes Influenciadas', 'Ordenes totales',]]
except Exception as ex: 
    print("Oh rallos y centellas, a ocurrido un error catastrofico! ", ex)
# df.groupby(['DISTRI', 'Periodo']).sum()
# df.sample(10)

In [67]:
# df.drop(columns='DISTRI'=='OTRO DISTRI',axis=0)
# df
# df = df[df['DISTRI']!='OTRO DISTRI']
df.sample(3)

,Periodo,COD_DISTRI,COD_CLIENTE,Ordenes inf.Quick Order,Ordenes inf. Upsell,Ordenes Influenciadas,Ordenes totales
7042,2022-08-01,369117,8218,1,1,1,9
120849,2022-08-01,328660,6117,1,0,1,1
56910,2022-08-01,369846,349,2,4,4,4


In [68]:
def run():
    # try:
    #     engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(parameters))
    #     print("Conect succesfull")
    # except Exception:
    #     print("Errora in DataBase conection")
    try:    
        df.to_excel('y:/19-POWER BI/01-Total Bebidas/TACO GQ/Algoritmo_Agosto_procesado.xlsx')
        df.to_sql('BEES_Algoritmo', con = enginex, if_exists= 'append', index=False, dtype={'Periodo': DATE})
        print("carga exitosa")
    except Exception:
        print("Error fatal irrecuperable")
text=("""DELETE FROM BEES_Algoritmo
    WHERE Periodo = '2022-08-01'""")
enginex.execute(text)
run()

carga exitosa
